In [327]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [328]:
# settings
np.random.seed(2020)
name = "Dblp"
d = 128
come_model_type = "GMM"
ks = [2, 5, 10]

In [329]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [330]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


k = 2
k = 5
k = 10


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.378200,-0.568244,-0.321310,0.652913,0.362287,-0.022258,0.083941,0.478697,-0.054290,0.311082,...,0.088833,-0.094781,-0.520394,-0.142101,-0.319075,0.165983,0.012775,-0.261246,0,0
1,0.325947,-0.507076,-0.239915,0.638949,0.218147,-0.146106,0.159716,0.465823,-0.221986,0.491792,...,-0.009053,-0.112342,-0.518923,0.030936,-0.338719,0.084743,0.093427,-0.121202,0,2
2,0.449000,-0.457566,-0.349665,0.697596,0.393687,0.025347,0.075684,0.514988,0.042532,0.333629,...,0.068656,0.038768,-0.541039,-0.147945,-0.326729,0.110152,-0.006950,-0.237618,0,2
3,0.384735,-0.512173,-0.314645,0.580443,0.286935,-0.132966,0.082575,0.452176,-0.001800,0.294977,...,0.012493,-0.113152,-0.464554,-0.150216,-0.179702,0.151236,0.086842,-0.239395,0,0
4,0.302965,-0.452777,-0.367516,0.644588,0.318358,-0.004250,-0.035937,0.425295,0.139306,0.280322,...,0.129463,-0.123344,-0.528231,-0.157348,-0.234854,0.134431,-0.012346,-0.262052,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.279475,-0.413140,-0.188146,0.484287,0.390216,-0.066214,0.053672,0.464491,0.020508,0.332705,...,-0.138939,-0.014533,-0.396638,-0.149566,0.034779,0.140223,0.148806,-0.191780,0,0
13180,-0.060374,-0.451956,-0.251877,0.026295,0.297827,0.072846,0.150009,0.115362,0.182046,0.447072,...,-0.329179,0.355102,0.038367,0.288219,0.048088,-0.547430,0.167478,0.377548,1,3
13181,-0.144091,-0.375015,-0.103247,0.024451,0.269478,-0.036013,0.036947,0.045607,0.273113,0.499460,...,-0.047350,0.400932,-0.073496,0.112511,-0.111854,-0.536318,0.065888,0.445000,1,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.016328,-0.281995,-0.026873,0.284062,0.223244,-0.091031,0.183607,0.324336,0.064249,0.435182,...,-0.134908,0.221013,-0.266594,0.017478,0.027901,-0.032389,0.118213,0.145175,0.610058,2.190382
std,0.161661,0.180502,0.165921,0.195031,0.167036,0.175895,0.159877,0.179617,0.174731,0.143873,...,0.152300,0.158535,0.221948,0.138263,0.169343,0.197521,0.172907,0.239865,0.487755,1.315339
min,-0.571476,-0.921358,-0.523288,-0.372199,-0.364393,-0.675392,-0.469759,-0.368955,-0.496001,-0.252168,...,-0.726665,-0.402675,-0.922494,-0.574979,-0.549227,-0.720108,-0.485893,-0.433391,0.000000,0.000000
25%,-0.094218,-0.414355,-0.140715,0.147333,0.104092,-0.217716,0.082809,0.202268,-0.070563,0.344811,...,-0.233087,0.113991,-0.440425,-0.073056,-0.092345,-0.170864,0.000903,-0.053244,0.000000,1.000000
50%,0.010018,-0.281766,-0.043871,0.294969,0.227463,-0.098726,0.182552,0.325245,0.063575,0.441522,...,-0.123557,0.214511,-0.262143,0.012757,0.032562,-0.038203,0.096182,0.159942,1.000000,3.000000
75%,0.124693,-0.147574,0.075153,0.427007,0.338933,0.034184,0.288909,0.444196,0.195043,0.533912,...,-0.033907,0.324983,-0.086340,0.105599,0.150699,0.113821,0.219016,0.328873,1.000000,3.000000
max,0.694223,0.352552,0.691326,0.958631,0.853563,0.483939,0.730213,1.082313,0.676577,0.917763,...,0.426009,0.838479,0.383111,0.571130,0.514615,0.622779,0.838232,0.907394,1.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.373667,-0.598349,-0.327014,0.657728,0.351909,0.038550,-0.080115,0.461961,-0.016757,0.173869,...,0.188289,-0.088379,-0.321589,-0.028481,-0.349754,-0.005408,-0.034814,-0.265680,2,0
1,0.288232,-0.558893,-0.225592,0.581605,0.172607,-0.036074,-0.040925,0.300119,-0.119094,0.388998,...,0.142187,-0.148034,-0.350271,0.022388,-0.463542,-0.074803,0.153513,-0.077299,2,2
2,0.522046,-0.481964,-0.352141,0.545657,0.362342,0.059359,-0.125334,0.532152,0.220742,0.267989,...,0.130193,0.021059,-0.395705,-0.101943,-0.387372,-0.119173,-0.011038,-0.241888,2,2
3,0.444355,-0.533085,-0.235819,0.507681,0.305394,-0.070556,-0.077944,0.293856,0.043477,0.111802,...,0.129462,-0.115245,-0.214651,-0.118961,-0.181251,-0.015813,0.119510,-0.246060,2,0
4,0.259767,-0.518458,-0.251856,0.533683,0.188223,0.182554,-0.312260,0.317080,0.202606,0.166693,...,0.263655,-0.146313,-0.376286,-0.241984,-0.379285,-0.127399,-0.018786,-0.226779,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.284492,-0.427115,-0.191759,0.341836,0.327479,0.092717,-0.096083,0.346453,0.141263,0.281167,...,-0.052491,-0.184895,-0.191674,-0.257070,-0.071092,-0.055017,0.320599,-0.043624,2,0
13180,-0.014233,-0.478577,-0.277484,-0.024719,0.373877,0.033069,0.107526,0.051211,0.254965,0.522942,...,-0.270520,0.294522,0.116406,0.200431,0.009214,-0.535163,0.119168,0.482034,4,3
13181,-0.082071,-0.420717,-0.131762,0.030767,0.366723,-0.060749,0.005799,-0.007813,0.386638,0.543871,...,-0.038265,0.346535,0.049381,0.032367,-0.122283,-0.513771,0.051255,0.568714,4,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.011870,-0.274936,-0.041257,0.210682,0.197878,-0.095127,0.129304,0.269120,0.064481,0.402140,...,-0.094202,0.175426,-0.180831,-0.006417,0.022129,-0.068284,0.111088,0.178563,2.364078,2.190382
std,0.175686,0.197672,0.197935,0.181454,0.181297,0.213848,0.186193,0.195496,0.207028,0.161858,...,0.178790,0.179498,0.209872,0.170388,0.186436,0.188960,0.179849,0.244257,1.562242,1.315339
min,-0.636390,-0.945618,-0.638133,-0.436039,-0.440479,-0.710476,-0.567830,-0.453427,-0.505427,-0.305486,...,-0.812171,-0.504598,-0.888819,-0.727660,-0.588370,-0.798604,-0.518078,-0.591128,0.000000,0.000000
25%,-0.106865,-0.419505,-0.173731,0.089501,0.072923,-0.249365,0.003479,0.134541,-0.095135,0.296907,...,-0.202578,0.054040,-0.323110,-0.118956,-0.115119,-0.190931,-0.008723,-0.006844,1.000000,1.000000
50%,0.003471,-0.280818,-0.063732,0.210753,0.190296,-0.112949,0.133000,0.263528,0.067965,0.410188,...,-0.075804,0.168957,-0.159624,-0.025157,0.019737,-0.070620,0.092221,0.163120,3.000000,3.000000
75%,0.127705,-0.127094,0.071763,0.327651,0.313916,0.053425,0.262387,0.397421,0.218109,0.513436,...,0.028952,0.290766,-0.028902,0.095018,0.151714,0.059513,0.212744,0.361721,4.000000,3.000000
max,0.723513,0.426340,0.849633,1.033853,0.959477,0.617446,0.744828,1.033402,0.732793,1.122846,...,0.465347,0.834750,0.436893,0.709512,0.664208,0.570490,0.915631,0.975744,4.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.417091,-0.660605,-0.272473,0.443058,0.343344,0.061323,-0.008461,0.358755,-0.038639,0.241695,...,0.118625,-0.180954,-0.319620,-0.087934,-0.445735,0.040443,0.094593,-0.216180,7,0
1,0.350649,-0.605619,-0.220139,0.447965,0.202301,-0.083260,0.127779,0.350898,-0.219289,0.312048,...,0.101710,-0.234587,-0.401559,0.064026,-0.540255,0.004373,0.268322,-0.031753,7,2
2,0.539922,-0.589285,-0.322726,0.533756,0.349594,0.076302,-0.084559,0.418261,0.023980,0.244881,...,0.141924,-0.092511,-0.282722,-0.130106,-0.427061,0.003204,0.079664,-0.192690,7,2
3,0.545547,-0.586747,-0.241170,0.372913,0.188718,-0.105797,-0.029614,0.224991,0.089034,0.110768,...,0.003011,-0.276135,-0.262280,-0.195758,-0.303646,0.113557,0.159748,-0.234632,7,0
4,0.359861,-0.589746,-0.277263,0.486219,0.279290,0.011161,-0.249149,0.271716,0.233651,0.139917,...,0.128323,-0.200226,-0.380227,-0.070451,-0.330802,0.056905,0.030870,-0.178824,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.285899,-0.640184,-0.051463,0.262650,0.365091,-0.068179,-0.149960,0.340737,0.124058,0.263513,...,-0.096764,-0.121576,-0.098048,-0.170548,-0.137315,-0.013470,0.331974,-0.104975,7,0
13180,0.035048,-0.401313,-0.201068,-0.053171,0.422753,0.233010,-0.077103,0.041496,0.125276,0.553925,...,-0.162384,0.318242,0.122423,0.337378,0.122951,-0.556136,0.191998,0.355595,9,3
13181,-0.033802,-0.314324,-0.069181,-0.145071,0.457281,0.153484,-0.065174,-0.074060,0.223868,0.484827,...,0.032495,0.299399,0.056879,0.181140,0.036609,-0.440997,0.218466,0.354365,9,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.033823,-0.274015,-0.020886,0.237328,0.181031,-0.070742,0.135394,0.238457,0.071768,0.396109,...,-0.093074,0.157598,-0.156266,-0.010030,0.026143,-0.080903,0.115483,0.164446,4.524499,2.190382
std,0.197209,0.212052,0.194632,0.192579,0.194326,0.214966,0.193277,0.204405,0.204465,0.176717,...,0.183707,0.195677,0.203245,0.185779,0.191305,0.196750,0.198647,0.218512,2.747870,1.315339
min,-0.570941,-1.048615,-0.675844,-0.424649,-0.442650,-0.779538,-0.555697,-0.488389,-0.546452,-0.378798,...,-0.865245,-0.497878,-0.933111,-0.707183,-0.590624,-0.854635,-0.533849,-0.572916,0.000000,0.000000
25%,-0.104158,-0.420021,-0.155895,0.103159,0.043123,-0.224035,0.001714,0.097753,-0.084192,0.285592,...,-0.209700,0.026983,-0.293739,-0.139719,-0.120186,-0.211021,-0.027397,0.004894,2.000000,1.000000
50%,0.022862,-0.264023,-0.041408,0.238433,0.176651,-0.086057,0.136090,0.229827,0.062583,0.402700,...,-0.077697,0.151573,-0.141944,-0.019980,0.021078,-0.084601,0.100352,0.156233,5.000000,3.000000
75%,0.169107,-0.123467,0.099126,0.367766,0.308192,0.075537,0.268212,0.369611,0.218240,0.519076,...,0.033205,0.281722,-0.011461,0.106539,0.166452,0.045583,0.239779,0.320728,7.000000,3.000000
max,0.780385,0.492900,0.811352,0.998509,0.958092,0.669904,0.711403,1.003308,0.804468,0.960492,...,0.557454,0.929424,0.443088,0.725085,0.648958,0.671382,0.938668,0.927319,9.000000,4.000000


# Community Detection

In [331]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.02982812550385347
  i1 conductance: 0.02982812550385347
=>conductance: 0.02982812550385347
K = 5
  i0 conductance: 0.2666713512753847
  i1 conductance: 0.055547663020315385
  i2 conductance: 0.08794559099437148
  i3 conductance: 0.011954077405610131
  i4 conductance: 0.07281137570669864
=>conductance: 0.011954077405610131
K = 10
  i0 conductance: 0.30562937907707177
  i1 conductance: 0.26101694915254237
  i2 conductance: 0.024672675833936442
  i3 conductance: 0.38368943725549204
  i4 conductance: 0.25575923929026567
  i5 conductance: 0.3694458312531203
  i6 conductance: 0.1797880662826257
  i7 conductance: 0.1377079482439926
  i8 conductance: 0.08507386653082018
  i9 conductance: 0.37375
=>conductance: 0.024672675833936442


In [332]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.43201662871155316
K = 5
NMI:  0.6759869880106791
K = 10
NMI:  0.45463103916973724


# Node Classification

In [333]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0,128) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.2f}")
    print(f"  macro_f1: {macro_f1*100:.2f}")

k = 2
  micro_f1: 92.57
  macro_f1: 92.62
k = 5
  micro_f1: 92.49
  macro_f1: 92.32
k = 10
  micro_f1: 93.18
  macro_f1: 92.92
